In [2]:
# h1: first haplotype
# h2: second haplotype
# s: effect size of variants
gene_expression_estimation<-function(h1,h2,s){
  log_expression_h1<-t(h1)%*%s
  log_expression_h2<-t(h2)%*%s
  total_expression<-exp(log_expression_h1)+exp(log_expression_h2)
  expression_ratio_h1<-exp(log_expression_h1)/total_expression
  expression_ratio_h2<-exp(log_expression_h2)/total_expression
  result<-c(log_expression_h1,log_expression_h2,total_expression,expression_ratio_h1,expression_ratio_h2)
  return(result)
  
}

# producing all genotypes
number2binary = function(number, noBits) {
       binary_vector = rev(as.numeric(intToBits(number)))
       if(missing(noBits)) {
          return(binary_vector)
       } else {
          binary_vector[-(1:(length(binary_vector) - noBits))]
       }
    }

#lookup table for n variants

AFC=read.table(file="aFC_Whole_Blood.txt", header=TRUE, sep="\t")
# maximum number of variants
maximum_variants <- 16

for (var_count in 1:maximum_variants){    
    lookup_table<-data.frame()
    lookup_table_index<-1
    count<-1
    result<-c()
    count_result<-c()
    for (index in 1:nrow(AFC)){
        
        # check the number of variants 
        current_gene<-paste(AFC[index,'gene_id'])
        print (current_gene)
        next_gene<-paste(AFC[index+1,'gene_id'])
        
        if(identical(current_gene,next_gene)){
            count<-count+1
            
            }
        else{
            
            if (var_count==count){
                    
                gene_id<-gsub("\\..*","",AFC[index,'gene_id'])
                lookup_table[lookup_table_index,'gene_id']<-paste0(gene_id)
                AFC_vector<-c()
                var_index<-1
                    for (temp_index in (index-count+1):index){
                        AFC_value<-AFC[temp_index,'log2_aFC']
                        
                        
                        
                        if (AFC_value=='NaN'| AFC_value=='nan' | is.na(AFC_value))
                            AFC_value<-0
                        
                        
                        # Cut off afc values
                        if (AFC_value < -6.64){
                            AFC_value <- -log2(100)
                            }
                        if (AFC_value > 6.64){
                            AFC_value <- log2(100)
                            }
                        # Cut off afc values
                        
                        AFC_vector<- c(AFC_vector,AFC_value)
                        
                        # create lookup table
                        
                        lookup_table[lookup_table_index,paste0('variant_id_',var_index)]<-paste0(AFC[temp_index,'variant_id'])

                        lookup_table[lookup_table_index,paste0('rank_',var_index)]<-paste0(AFC[temp_index,'rank'])
                        var_index<-var_index+1    
                        }
                    for (j in 0:(2^count-1)){
                        
                        ref_alt_vector<-c(number2binary(j,count))
                        ref_vector<-c(replicate(count,0))
                        output<-gene_expression_estimation(ref_alt_vector,ref_vector,AFC_vector)
                        lookup_table[lookup_table_index,paste0("",toString(ref_alt_vector))]<-paste0(output[1])
                        
                        
                        
                    }
                   
                    lookup_table_index<-lookup_table_index+1   
                
                
                
                }
            count<-1
            }#end of else
        }
# create output  
    
    write.csv(lookup_table, file=paste0("output",var_count,".csv"))
    write.table(lookup_table,file=paste0("output",var_count,".txt"))
    
    
    }

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

